In [1]:
# 导入必要的库
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os
api_key = os.getenv("OPENAI_API_KEY")
api_base = 'https://api.yesapikey.com/v1'  # 替换为你的 API 端点
# 使用 WebBaseLoader 从网页加载内容，并仅保留标题、标题头和文章内容
bs4_strainer = bs4.SoupStrainer(class_=("dtitle", "dinfo", "abstract", "acontent", "afooter", "side"))
loader = WebBaseLoader(
    web_paths=("https://news.cqu.edu.cn/archives/medias/content/2020/06/18/29da9bc33b146db5b36af2908861e1de88c25fc8.html",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

# 检查加载的文档内容长度
print(len(docs[0].page_content))  # 打印第一个文档内容的长度
# 查看第一个文档（前100字符）
print(docs[0].page_content[:100])

# 使用 RecursiveCharacterTextSplitter 将文档分割成块，每块1000字符，重叠200字符
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

# 检查分割后的块数量和内容
print(len(all_splits))  # 打印分割后的文档块数量
print(len(all_splits[0].page_content))  # 打印第一个块的字符数
print(all_splits[0].page_content)  # 打印第一个块的内容
print(all_splits[0].metadata)  # 打印第一个块的元数据

2119
【光明日报】一对教授夫妇的乡村振兴图谱

光明日报作者 :   张国圣




阅读 :  0





                   摘要
                

       
5
175
【光明日报】一对教授夫妇的乡村振兴图谱

光明日报作者 :   张国圣




阅读 :  0





                   摘要
                

                    6月18日，《光明日报》头版报道我校建筑城规学院杨宇振、覃琳夫妇致力于乡村振兴，在脱贫攻坚道路上体现知识分子责任担当的故事。
{'source': 'https://news.cqu.edu.cn/archives/medias/content/2020/06/18/29da9bc33b146db5b36af2908861e1de88c25fc8.html', 'start_index': 0}


In [2]:
# 使用 Chroma 向量存储和 OpenAIEmbeddings 模型，将文本转为向量，将分割的文档块嵌入并存储
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OpenAIEmbeddings(openai_api_key=api_key,
    openai_api_base=api_base)
)

In [3]:
# 查看 vectorstore 数据类型
type(vectorstore)

langchain_chroma.vectorstores.Chroma

In [4]:
# 使用 VectorStoreRetriever 从向量存储中检索与查询最相关的文档
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [5]:
type(retriever)

langchain_core.vectorstores.VectorStoreRetriever

In [6]:
retrieved_docs = retriever.invoke("农村是怎么建设的?")

Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


In [7]:
# 检查检索到的文档内容
print(len(retrieved_docs))  # 打印检索到的文档数量

5


In [8]:
print(retrieved_docs[0].page_content)  # 打印第一个检索到的文档内容

【脱贫攻坚路上，知识分子这样担当】 　　杨宇振和覃琳夫妇是重庆大学建筑城规学院的教师，近几年，他俩跑乡村的时间越来越多，积极性也越来越高。原因很简单，他们认为“机会来了”。 　　杨宇振和覃琳最早从专业角度思考“农村是怎么建设的”，是从重庆的少数民族建筑开始的。2000年到2004年，覃琳连续做了4年土家族建筑田野调查。“不管去哪里，每次都尽量在老乡家借宿，体验他们是怎么设计、怎么使用房屋的。”覃琳说。跑农村的次数多了，他们熟悉了重庆民族地区民居和公共建筑的分布。 　　参加重庆建筑师、规划师、工程师“三师下乡”活动后，杨宇振和覃琳与乡村的联系更密切了。刚开始主要是自己做一些研究，在相关会议上提提意见建议。此后，他俩把这些研究串起来，“现代化建设过程中如何面对传统建筑风貌”的思路也越来越清晰。“新一轮乡村建设正在推进，这对我们来说确实是难得的机遇。”杨宇振说。 　　2006年，两人明确了主要的发展方向，他们希望“在大山大水的大格局中”做研究，希望能够让那些在扶贫开发和乡村振兴过程中发展起来的场镇和村落，更好地契合“山的走向、水的流向”。“机会来了，我们得努力抓住。”这是杨宇振和覃琳的共识。 　　他们将自己关于乡村建筑的理想，更多地融入新兴的场镇和村居中去，让规划设计既能发掘、理解、体现当地的历史和文化传统，又能满足当下的现实需求。在一个乡村的规划设计中，两人将一处建筑设计成全村的共享空间，回访时却发现那里经常锁着门；为一个村民活动中心设计了圆拱门，却发现施工队怎么修都修不圆；规划设计中借用原有的两棵树作景观树，树的主人知道后却表示如果不给钱就要把树砍掉。这些都是教研室和课堂上不会遇到的问题，杨宇振和覃琳带着一批批学生去现场参与协调处理，又把这些问题、素材和图片带回来，让更多的学生理解一个乡村建筑到底是怎么修起来的，为什么会这么修，理解“在乡村做建设就要理解乡村的状态，理解技术上的可行性”。 　　他们更重视为什么人做设计、什么人将使用自己的设计成果，“这是一个社会学的问题，一个专业人员要有自己的理想和坚持，承担自己应该承担的责任”。这也促使他们总是希望能让乡村的公共建筑，真正成为一个能够低成本维护的完全开放的公共空间。


In [9]:
# 定义 RAG 链，将用户问题与检索到的文档结合并生成答案
llm = ChatOpenAI(openai_api_key=api_key,
    openai_api_base=api_base, model="gpt-4o-mini")

In [10]:
# 使用 hub 模块拉取 rag 提示词模板
prompt = hub.pull("rlm/rag-prompt")

D:\pythonPaChong\openai-quickstart\venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [11]:
# 打印模板
print(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [12]:
# 为 context 和 question 填充样例数据，并生成 ChatModel 可用的 Messages
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

In [13]:
# 查看提示词
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [14]:
# 定义格式化文档的函数
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [15]:
# 使用 LCEL 构建 RAG Chain
rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)

In [16]:
# 流式生成回答
for chunk in rag_chain.stream("农村是怎么建设的?"):
    print(chunk, end="", flush=True)

Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


农村建设通常涉及结合当地的历史文化与现代需求，同时考虑生态和公共服务的优化。教授夫妇杨宇振和覃琳通过实地调研，设计出适应扶贫和发展的场镇规划，以保留原有环境和氛围。乡村建设的过程中，他们强调理解乡村状态和技术可行性，并致力于创建低成本维护的公共空间。

In [17]:
# 流式生成回答
for chunk in rag_chain.stream("三师下乡?"):
    print(chunk, end="", flush=True)

Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


“三师下乡”是指建筑师、规划师和工程师等专业人员深入农村，参与乡村建设和振兴活动。杨宇振和覃琳夫妇通过这一活动，加强了与乡村的联系，并在扶贫和乡村振兴中贡献自己的专业知识。该活动旨在提升乡村公共建筑的功能和可持续性，促进乡村的发展。

In [18]:
# 流式生成回答
for chunk in rag_chain.stream("讲了什么故事?"):
    print(chunk, end="", flush=True)

Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


故事讲述了重庆大学的教授夫妇杨宇振和覃琳如何致力于乡村振兴，积极参与乡村建设与规划，以应对扶贫和发展的挑战。他们通过调研和实践，关注乡村建筑的历史与文化，努力将设计理念与当地实际相结合，提升公共空间的可用性与可维护性。二人深感乡村建设的责任与乐趣，致力于推动乡村的可持续发展。

In [19]:
prompt1 = hub.pull("pisles/rag-prompt-llama-ps")
# 使用 LCEL 构建 RAG Chain
rag_chain1 = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt1
        | llm
        | StrOutputParser()
)
# 流式生成回答
for chunk in rag_chain1.stream("农村是怎么建设的?"):
    print(chunk, end="", flush=True)



D:\pythonPaChong\openai-quickstart\venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)
Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


农村建设是通过结合当地的历史、文化和自然环境，进行规划和设计，从而实现现代化与传统的协调。杨宇振和覃琳夫妇在重庆的乡村建设中，强调了生态和公共服务的重要性，力求创造开放、可维护的公共空间。他们的实践经验表明，理解乡村的真实状态和居民需求是成功建设的关键。

In [21]:
# 流式生成回答
for chunk in rag_chain1.stream("三师下乡?"):
    print(chunk, end="", flush=True)

Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


“三师下乡”是指建筑师、规划师和工程师等专业人士深入乡村，参与乡村建设和发展，旨在提升农村的基础设施和公共服务。这一活动促进了城市与乡村之间的联系，帮助专业人员理解和应对乡村建设中的实际问题。在重庆大学的杨宇振和覃琳夫妇的案例中，他们通过这一活动，积极参与乡村振兴和脱贫攻坚，展现了知识分子的责任担当。

In [22]:
for chunk in rag_chain1.stream("讲了什么故事?"):
    print(chunk, end="", flush=True)

Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


这篇文章讲述了重庆大学教授夫妇杨宇振和覃琳在乡村振兴与脱贫攻坚中发挥知识分子责任的故事。他们通过参与乡村建筑的规划与设计，努力将现代建筑与当地文化和环境相结合，推动乡村的发展与建设。文章还提到他们在实践中遇到的挑战与解决方案，以及对乡村变化的深刻感悟。

In [ ]:
from langchain_core.prompts import PromptTemplate

# 自定义提示词模板
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

In [ ]:
# 为 context 和 question 填充样例数据，生成 LLM 可用的提示词
print(custom_rag_prompt.invoke({"context": "filler context", "question": "filler question"}).text)

In [ ]:
# 重新自定义 RAG Chain
custom_rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | custom_rag_prompt
        | llm
        | StrOutputParser()
)
# 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
# 使用自定义 prompt 生成回答
custom_rag_chain.invoke("乡村建筑的理想?")